In [1]:
import pandas as pd
from ydata_profiling import ProfileReport

In [10]:
df = pd.read_csv("Week5/ds_salaries.csv", index_col=0)
print(df)

     work_year experience_level  ... company_location company_size
0         2020               MI  ...               DE            L
1         2020               SE  ...               JP            S
2         2020               SE  ...               GB            M
3         2020               MI  ...               HN            S
4         2020               SE  ...               US            L
..         ...              ...  ...              ...          ...
602       2022               SE  ...               US            M
603       2022               SE  ...               US            M
604       2022               SE  ...               US            M
605       2022               SE  ...               US            M
606       2022               MI  ...               US            L

[607 rows x 11 columns]


In [11]:
profile = ProfileReport(df, title="Salaries report", explorative=True)
profile.to_file("salaries_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# Task 1:Job title nào có mức lương trung bình theo USD cao nhất
job_stats = df.groupby('job_title').agg({
    'salary_in_usd': ['mean', 'count', 'std']
    }).round(2)
job_stats.columns = ['avg_salary', 'count', 'std_salary']
job_stats_sorted = job_stats.sort_values('avg_salary', ascending=False)
print(job_stats_sorted.head(10))

                          avg_salary  count  std_salary
job_title                                              
Data Analytics Lead        405000.00      1         NaN
Principal Data Engineer    328333.33      3   235389.75
Financial Data Analyst     275000.00      2   247487.37
Principal Data Scientist   215242.43      7    94705.04
Director of Data Science   195074.00      7    70015.79
Data Architect             177873.91     11    45714.72
Applied Data Scientist     175655.00      5   123647.19
Analytics Engineer         175000.00      4    29508.08
Data Specialist            165000.00      1         NaN
Head of Data               160162.60      5    88780.42


In [15]:
# Task 2:Data Scientist mang quốc tịch nào có mức lương trung bình cao nhất
ds_df = df[df['job_title'] == 'Data Scientist']
avg_salary_by_country = ds_df.groupby('employee_residence').agg({
    'salary_in_usd': ['mean', 'count']
}).round(2).sort_values(('salary_in_usd', 'mean'), ascending=False)
print(avg_salary_by_country.head(10))

                   salary_in_usd      
                            mean count
employee_residence                    
US                     149408.33    78
CH                     122346.00     1
SG                     119059.00     1
DZ                     100000.00     1
AT                      91237.00     1
GB                      88177.36    11
AU                      86703.00     1
CA                      81489.17     6
DE                      74773.71     7
GR                      68428.00     1


In [19]:
# Task 3:Có phải ở USA, nhân sự ngoại quốc nhận được mức lương trung bình cao hơn nhân sự trong nước không
us_companies = df[df['company_location'] == 'US']
us_salaries = us_companies.groupby(us_companies['employee_residence'] == 'US').salary_in_usd.agg([
    'mean',
    'count'
]).round(2)

us_salaries.index = ['Foreign Workers', 'US Workers']

print("So sánh lương trung bình tại công ty Mỹ:")
print(us_salaries)

So sánh lương trung bình tại công ty Mỹ:
                      mean  count
Foreign Workers   79366.84     25
US Workers       148955.90    330


In [44]:
# Task 4: Mức lương trung bình của Data Engineer (SE level) ở các công ty nhỏ và vừa ở USA
filtered_df = df[
    df['job_title'].str.contains('Data Engineer') &
    (df['experience_level'] == 'SE') &
    (df['company_location'] == 'US') &
    (df['company_size'].isin(['S', 'M']))
]

# Tính lương trung bình theo size công ty
avg_salary = filtered_df.groupby('job_title').agg({
    'salary_in_usd': ['mean', 'count']
}).round(2)

print(avg_salary)

                        salary_in_usd      
                                 mean count
job_title                                  
Cloud Data Engineer         160000.00     1
Data Engineer               140680.19    47
Lead Data Engineer          160000.00     1
Principal Data Engineer     200000.00     1


In [23]:
# Task 5: So sánh mức lương của Data Scientist ở USA ở các mức trình độ (experience_level) khác nhau
filtered_df = df[
    (df['job_title'] == 'Data Scientist') &
    (df['company_location'] == 'US')
]

avg_salary = filtered_df.groupby('experience_level').agg({
    'salary_in_usd': ['mean', 'count']
})

print(avg_salary)

                  salary_in_usd      
                           mean count
experience_level                     
EN                 88833.333333     6
MI                112547.560000    25
SE                163679.773585    53


In [26]:
# Task 6: Nhân sự là người nước ngoài chiếm tỉ lệ bao nhiêu phần trăm ở các công ty lớn ở USA
filtered_df = df[
    (df['company_location'] == 'US') &
    (df['company_size'] == 'L')
]

total_employees = len(filtered_df)
foreign_employees = len(filtered_df[filtered_df['employee_residence'] != 'US'])

foreign_percentage = round((foreign_employees / total_employees * 100), 2)

print(f"{foreign_percentage}%")

7.55%


In [28]:
# Task 7: Tìm ra 3 job title có mức lương trung bình cao nhất ở level MI dành cho nhân sự trong nước
filtered_df = df[
    (df['company_location'] == 'US') &
    (df['experience_level'] == 'MI')
]

avg_salary = filtered_df.groupby('job_title').agg({
    'salary_in_usd': ['mean', 'count']
}).round(2).sort_values(('salary_in_usd', 'mean'), ascending=False)
print(avg_salary.head(10))

                                   salary_in_usd      
                                            mean count
job_title                                             
Research Scientist                     450000.00     1
Financial Data Analyst                 450000.00     1
ML Engineer                            270000.00     1
Data Science Manager                   200000.00     2
Applied Machine Learning Scientist     178800.00     3
Data Architect                         166666.67     3
AI Scientist                           160000.00     2
Applied Data Scientist                 157000.00     1
Principal Data Scientist               151000.00     1
Machine Learning Scientist             141766.67     3


In [30]:
# Task 8: Job nào ở Germany (DE) có mức lương trung bình thấp nhất
filtered_df = df[
    (df['employee_residence'] == 'DE')
]

avg_salary = filtered_df.groupby('job_title').agg({
    'salary_in_usd': ['mean', 'count']
}).round(2).sort_values(('salary_in_usd', 'mean'), ascending=False)
print(avg_salary.head(1))

                         salary_in_usd      
                                  mean count
job_title                                   
Principal Data Scientist     161565.67     3


In [45]:
# Task 9: Job nào có khoảng cách thu nhập giữa SE và MI ít nhất
avg_salary = df.groupby(['job_title', 'experience_level'])['salary_in_usd'].mean()
print(avg_salary)

# Tạo DataFrame với cột SE và MI
salary_comparison = pd.DataFrame({
    'SE': avg_salary.xs('SE', level=1),
    'MI': avg_salary.xs('MI', level=1)
}).dropna()  # Loại bỏ các job không có đủ cả SE và MI

salary_comparison['gap'] = abs(salary_comparison['SE'] - salary_comparison['MI'])
result = salary_comparison.sort_values('gap')

print(result.head().round(2))

job_title                      experience_level
3D Computer Vision Researcher  MI                    5409.000000
AI Scientist                   EN                   21987.250000
                               MI                  160000.000000
                               SE                   55000.000000
Analytics Engineer             EX                  155000.000000
                                                       ...      
Product Data Analyst           MI                   13036.000000
Research Scientist             EN                   86132.000000
                               MI                  136498.142857
                               SE                   88859.400000
Staff Data Scientist           SE                  105000.000000
Name: salary_in_usd, Length: 105, dtype: float64
                                   SE         MI       gap
job_title                                                 
Machine Learning Developer   78791.00   78791.00      0.00
Lead Data S

In [54]:
# Task 10: Job nào có mức lương trung bình đều nằm trong top 5 ở tất cả các level
avg_salary = df.groupby(['job_title', 'experience_level'])['salary_in_usd'].mean()

levels = df['experience_level'].unique()
# levels = ['MI','SE']

top_jobs = []
for level in levels:
    filtered_df = df.loc[df.experience_level == level].groupby(['job_title'])['salary_in_usd'].mean().sort_values(ascending=False).head(5)
    top_jobs.append(filtered_df.index.to_list())
print(top_jobs)

import numpy as np
from functools import reduce
reduce(np.intersect1d, (top_jobs))

[['Financial Data Analyst', 'Data Science Manager', 'Applied Machine Learning Scientist', 'Data Architect', 'AI Scientist'], ['Data Analytics Lead', 'Applied Data Scientist', 'ML Engineer', 'Machine Learning Scientist', 'Machine Learning Infrastructure Engineer'], ['Machine Learning Scientist', 'Applied Data Scientist', 'Computer Vision Software Engineer', 'Financial Data Analyst', 'Machine Learning Developer'], ['Principal Data Engineer', 'Principal Data Scientist', 'Data Engineer', 'Head of Data', 'Director of Data Science']]


array([], dtype='<U40')

In [40]:
# Task 11: Job nào có mức chênh lệch thu nhập trung bình giữa EN & EX level lớn nhất (chỉ tính các công ty nhỏ và vừa)
filtered_df = df[
    (df['company_size'].isin(['S', 'M']))
]
avg_salary = filtered_df.groupby(['job_title', 'experience_level'])['salary_in_usd'].mean()

salary_comparison = pd.DataFrame({
    'EN': avg_salary.xs('EN', level=1),
    'EX': avg_salary.xs('EX', level=1)
}).dropna()

salary_comparison['gap'] = abs(salary_comparison['EN'] - salary_comparison['EX'])
result = salary_comparison.sort_values('gap', ascending=False)

print(result.head().round(2))

                               EN        EX        gap
job_title                                             
Data Engineer            58691.50  245500.0  186808.50
Data Analyst             48452.80  120000.0   71547.20
Data Science Consultant  57513.33   69741.0   12227.67


In [76]:
# Task 12: Nước nào trả lương cao nhất cho nhân sự nước ngoài
filtered_df = df[df['company_location'] != df['employee_residence']]
avg_salary = filtered_df.groupby(['company_location','employee_residence'])['salary_in_usd'].mean().sort_values(ascending=False).head(1)
print(avg_salary)

company_location  employee_residence
CA                US                    225000.0
Name: salary_in_usd, dtype: float64


In [56]:
# Task 13: Có phải với cùng experience_level, các công ty quy mô càng lớn thì trả lương càng cao không
filtered_df = df.groupby(['experience_level','company_size'])['salary_in_usd'].mean().sort_values(ascending=False).head(10)
print(filtered_df)

experience_level  company_size
EX                L               221942.181818
                  S               201309.333333
                  M               178241.750000
SE                L               147591.013889
                  M               137815.596774
                  S               116026.727273
MI                L                98030.372093
                  M                90091.081633
EN                L                72813.241379
                  S                62185.310345
Name: salary_in_usd, dtype: float64


In [ ]:
# Task 14: Sắp xếp các job title theo thứ tự tăng dần về thu nhập trung bình
avg_salary = df.groupby(['job_title'])['salary_in_usd'].mean().sort_values(ascending=False)
print(avg_salary)

In [66]:
# Task 15: Liệt kê những job title có mức thu nhập trung bình cao nhất ở 1 quốc gia, nhưng lại nằm trong top 3 job có thu nhập trung bình thấp nhất ở 1 quốc gia khác
company_locations = df['company_location'].unique()
for company_location in company_locations:
    avg_salary = df[df['company_location'] == company_location].groupby(['job_title'])['salary_in_usd'].mean().sort_values(ascending=False).head(1)
    # print(company_location)

    company_locations = df['company_location'].unique()
    for location in company_locations:
        if location != company_location:
            avg_salary_by_country = df[df['company_location'] == location].groupby(['job_title'])['salary_in_usd'].mean().sort_values(ascending=True).head(3)
            if avg_salary.index[0] in avg_salary_by_country.index.to_list():
                print(f'Job = {avg_salary.index[0]}, top 1 in {company_location}, top 3 lowest in {location}')
                break

Job = Machine Learning Scientist, top 1 in JP, top 3 lowest in PK
Job = Data Scientist, top 1 in HU, top 3 lowest in FR
Job = Data Analytics Engineer, top 1 in PK, top 3 lowest in GB
Job = Research Scientist, top 1 in CN, top 3 lowest in NL
Job = Data Engineer, top 1 in GR, top 3 lowest in DE
Job = Machine Learning Engineer, top 1 in NL, top 3 lowest in JP
Job = Data Science Engineer, top 1 in MX, top 3 lowest in GR
Job = Machine Learning Scientist, top 1 in CA, top 3 lowest in PK
Job = Data Scientist, top 1 in AT, top 3 lowest in HU
Job = Data Scientist, top 1 in NG, top 3 lowest in HU
Job = Data Engineering Manager, top 1 in ES, top 3 lowest in DE
Job = Research Scientist, top 1 in PT, top 3 lowest in CN
Job = Machine Learning Engineer, top 1 in IT, top 3 lowest in JP
Job = Machine Learning Engineer, top 1 in HR, top 3 lowest in JP
Job = Data Scientist, top 1 in LU, top 3 lowest in HU
Job = Big Data Engineer, top 1 in RO, top 3 lowest in US
Job = Computer Vision Engineer, top 1 in BR